In [276]:
import pandas as pd
import numpy as np

In [289]:
#Import Datasets

#PV Output, change to PVWatts automatic based on inputs
PVOutput = pd.read_excel('/Users/ajames2/Desktop/Python/PVOutput.xlsx')

#Customer load
CustomerLoad = pd.read_excel('https://openei.org/datasets/dataset/399852d9-91a3-4dd7-86ba-73dd5388c994/'
                              'resource/c0608a74-1ef8-4583-a4cd-33f0c78e8478/download/lbnlb74electricity.xlsx')
    #Change to be an excel file: CustomerLoad = pd.read_excel('/Users/ajames2/Desktop/Python/CustomerLoad.xlsx')

In [298]:
#Input Data

#Storage
    #Fixed
storage_size_kW = float(250)
storage_capex = float(250000)

#Diesel Generator
    #Fixed
diesel_size_kW = float(250)
diesel_capex = float(100000)
diesel_fuel_cost =float(3.5)
diesel_price = float(.40) #temporary

    #Variable
    
#Electricity Price - TOU
    #00:01 - 11:00
    #11:01 - 15:00
    #15:01 - 24:00
    
#Electricity Price - Fixed

electricity_price = float(0.13)

In [291]:
#Clean datasets and convert to hourly

#Rename kWh columns
CustomerLoad.rename(columns={'Building 74 - kWh Total Electricity (kWh)':'Customer_Load'}, inplace=True)

#Convert to hourly
CustomerLoad['timestamp_hour'] = CustomerLoad['Timestamp'].apply(
    lambda x: pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
CustomerLoad.pivot_table(index=['timestamp_hour'], 
                         values='Customer_Load', aggfunc='sum').reset_index()
del CustomerLoad['Timestamp']

In [292]:
#Add electricity price, solar production, net customer load

EnergyMaster = CustomerLoad
EnergyMaster['electricity_price'] = electricity_price

#Add pv production
EnergyMaster['PV_Output'] = PVOutput

#post load pv production
EnergyMaster['PV_Output_Post_Load'] = (EnergyMaster['PV_Output']-EnergyMaster['Customer_Load']).clip_lower(0)

#post PV load
EnergyMaster['Customer_Load_Post_PV'] = (EnergyMaster['Customer_Load']-EnergyMaster['PV_Output']).clip_lower(0)

#storage SOC
EnergyMaster['Storage_SOC'] = EnergyMaster['PV_Output_Post_Load'].cumsum().clip(0,250)

#new customer load post storage
EnergyMaster['Storage_SOC_Post_Load'] = (EnergyMaster['Storage_SOC']-EnergyMaster['Customer_Load']).clip_lower(0)

In [297]:
#create randomized grid outages

#variables
days_per_week_grid_down = 3
hoursdown = 4
percent_downtime = days_per_week_grid_down / (7.0*24/hoursdown)

#calculations
EnergyMaster['grid_down_probability'] = EnergyMaster['Customer_Load'].apply(lambda x: np.random.random())
def grid_is_down(rand_num):
    if rand_num < percent_downtime:
        return 1
    else:
        return 0
EnergyMaster['grid_down'] = EnergyMaster['grid_down_probability'].apply(grid_is_down)

In [304]:
#calculate costs in scenarios

#diesel scenario 

SyntaxError: invalid syntax (<ipython-input-304-cac83b1be2b2>, line 5)